In [89]:
import numpy as np
import random
from google.colab import drive
import json
import time
import pandas as pd
import os
import sys
import librosa
import soundfile as sf
import xgboost

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
for speaker in os.listdir("/content/gdrive/MyDrive/nus-smc-corpus_48/"):
  if '.txt' not in speaker:
    for 

KENN
MCUR
JTAN
MPOL
MPUR
ADIZ
NJAT
PMAR
JLEE
SAMF
ZHIY
VKOW


In [29]:
read_df = pd.read_csv("/content/gdrive/MyDrive/nus-smc-corpus_48/ADIZ/read/09.txt", delimiter=" ", header=None, names=['start', 'end', 'phoneme'])

In [30]:
read_df

,start,end,phoneme
0,0.00,1.05,sil
1,1.05,1.21,l
2,1.21,1.32,ah
3,1.32,1.37,v
4,1.37,1.39,sil
...,...,...,...
412,37.61,37.76,y
413,37.76,37.80,uw
414,37.80,37.90,g
415,37.90,38.17,ow


In [16]:
audioS, srS = librosa.load("/content/gdrive/MyDrive/nus-smc-corpus_48/ADIZ/read/01.wav", sr=None)
spectrogramS = librosa.stft(audioS, n_fft=2048, hop_length=256, center=False, win_length=2048)
MS = abs(spectrogramS)
phaseS = spectrogramS/(MS + 2.2204e-16)

In [18]:
audioS.shape

(902484,)

In [19]:
srS

44100

In [25]:
audioS.shape[0]/read_df['end'][183]

44152.837573385514

In [24]:
read_df['end'][183]

20.44

In [26]:
srS*read_df['end'][183]

901404.0

In [27]:
spectrogramS.shape

(1025, 3518)

In [28]:
3518/20.44

172.11350293542074

In [31]:
audioS, srS = librosa.load("/content/gdrive/MyDrive/nus-smc-corpus_48/ADIZ/read/09.wav", sr=None)
spectrogramS = librosa.stft(audioS, n_fft=2048, hop_length=256, center=False, win_length=2048)
MS = abs(spectrogramS)
phaseS = spectrogramS/(MS + 2.2204e-16)

In [32]:
audioS.shape

(1717086,)

In [45]:
read_df

,start,end,phoneme
0,0.00,1.05,sil
1,1.05,1.21,l
2,1.21,1.32,ah
3,1.32,1.37,v
4,1.37,1.39,sil
...,...,...,...
412,37.61,37.76,y
413,37.76,37.80,uw
414,37.80,37.90,g
415,37.90,38.17,ow


In [50]:
audioS[:10]

array([ 0.0000000e+00, -3.0517578e-05, -1.2207031e-04, -1.2207031e-04,
       -1.5258789e-04, -1.8310547e-04, -1.8310547e-04, -2.7465820e-04,
       -1.5258789e-04, -1.2207031e-04], dtype=float32)

In [52]:
audioS[:int(read_df['end'].iloc[-1]*srS)].shape

(1716372,)

In [46]:
read_df['end'].iloc[-1]*srS

1716372.0

In [33]:
1717086/read_df['end'][416]

44118.34532374101

In [34]:
spectrogramS.shape

(1025, 6700)

In [35]:
6700/read_df['end'][416]

172.14799588900308

In [73]:
read_df

,start,end,phoneme
0,0.00,1.05,sil
1,1.05,1.21,l
2,1.21,1.32,ah
3,1.32,1.37,v
4,1.37,1.39,sil
...,...,...,...
412,37.61,37.76,y
413,37.76,37.80,uw
414,37.80,37.90,g
415,37.90,38.17,ow


In [75]:
for i in range(int(read_df['start'].iloc[0]*100), int(read_df['end'].iloc[0]*100)):
  print(i/100)

0.0
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99
1.0
1.01
1.02
1.03
1.04


In [37]:
os.listdir("/content/gdrive/MyDrive/nus-smc-corpus_48/")

['KENN',
 'MCUR',
 'JTAN',
 'MPOL',
 'MPUR',
 'ADIZ',
 'NJAT',
 'PMAR',
 'JLEE',
 'SAMF',
 'ZHIY',
 'VKOW',
 'README.txt']

In [38]:
speaker_list = ['KENN',
 'MCUR',
 'JTAN',
 'MPOL',
 'MPUR',
 'ADIZ',
 'NJAT',
 'PMAR',
 'JLEE',
 'SAMF',
 'ZHIY']

In [39]:
test_speaker = 'VKOW'

In [40]:
os.listdir("/content/gdrive/MyDrive/nus-smc-corpus_48/ADIZ/read/")

['01.wav',
 '09.txt',
 '18.txt',
 '13.txt',
 '01.txt',
 '09.wav',
 '13.wav',
 '18.wav']

In [93]:
audio_array = []
phoneme_array = []

In [94]:
for sp in speaker_list:
  for sig in os.listdir("/content/gdrive/MyDrive/nus-smc-corpus_48/"+str(sp)+"/read/"):
    if '.wav' in sig:
      song = sig.replace('.wav','')
      df = pd.read_csv("/content/gdrive/MyDrive/nus-smc-corpus_48/"+str(sp)+"/read/"+str(song)+".txt", delimiter=" ", header=None, names=['start', 'end', 'phoneme'])
      audio, sr = librosa.load("/content/gdrive/MyDrive/nus-smc-corpus_48/"+str(sp)+"/read/"+str(sig), sr=None)
      audio = audio[:int(df['end'].iloc[-1]*sr)]

      for idx, row in df.iterrows():
        for i in range(int(row['start']*100),int(row['end']*100)):
          reqd_audio = audio[int(i/100*sr):int((i+1)/100*sr)]
          spectrogram = librosa.stft(reqd_audio, n_fft=256, hop_length=256, center=False, win_length=256)
          M = abs(spectrogram)
          audio_array.append(M)
          phoneme_array.append(one_hot_dict[row['phoneme']])

In [88]:
np.array(audio_array)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


array([array([[4.37818989e-02, 2.27263779e-03, 6.79824650e-02, ...,
        1.31672069e-01, 8.69553164e-03, 1.34576499e-01],
       [4.56237309e-02, 2.99167782e-02, 4.00063768e-02, ...,
        6.41017258e-02, 3.29150818e-02, 8.72185230e-02],
       [6.19328246e-02, 4.12415601e-02, 1.51375495e-02, ...,
        3.25954147e-02, 5.17521128e-02, 4.31988910e-02],
       ...,
       [1.07144192e-03, 1.54478115e-03, 1.14233489e-03, ...,
        5.35920379e-04, 8.77336541e-04, 5.94387762e-04],
       [2.79499247e-04, 9.53289273e-04, 8.83245375e-04, ...,
        5.59472130e-04, 3.09915253e-04, 2.00566341e-04],
       [5.45251765e-04, 9.01728854e-05, 8.04201991e-04, ...,
        8.31322395e-04, 4.30581684e-04, 4.59553266e-04]], dtype=float32),
       array([[2.5039107e-02, 2.5378691e-02, 1.4193659e-02, ..., 6.9688283e-02,
        1.4669709e-02, 1.3805054e-01],
       [7.6325893e-02, 8.6030811e-02, 8.3960712e-02, ..., 2.8557595e-02,
        9.4662800e-02, 1.9121478e-01],
       [1.7654189e-01, 2.

In [87]:
audio_array[6].shape

(257, 15)

In [61]:
len(audio_array)

31380

In [68]:
set(phoneme_array)

{'aa',
 'ae',
 'ah',
 'ao',
 'aw',
 'ay',
 'b',
 'ch',
 'd',
 'dh',
 'eh',
 'er',
 'ey',
 'f',
 'g',
 'hh',
 'ih',
 'iy',
 'jh',
 'k',
 'l',
 'm',
 'n',
 'ng',
 'ow',
 'oy',
 'p',
 'r',
 's',
 'sh',
 'sil',
 't',
 'th',
 'uh',
 'uw',
 'v',
 'w',
 'y',
 'z',
 'zh'}

In [66]:
one_hot = np.eye(40)

In [70]:
for ph in enumerate(set(phoneme_array)):
  print(ph)

(0, 'iy')
(1, 'r')
(2, 'ih')
(3, 'd')
(4, 'v')
(5, 't')
(6, 'g')
(7, 'l')
(8, 'ah')
(9, 'hh')
(10, 's')
(11, 'z')
(12, 'n')
(13, 'w')
(14, 'ay')
(15, 'f')
(16, 'ey')
(17, 'm')
(18, 'y')
(19, 'ae')
(20, 'k')
(21, 'dh')
(22, 'ch')
(23, 'aa')
(24, 'b')
(25, 'ow')
(26, 'uw')
(27, 'aw')
(28, 'jh')
(29, 'oy')
(30, 'sil')
(31, 'er')
(32, 'zh')
(33, 'eh')
(34, 'p')
(35, 'uh')
(36, 'ng')
(37, 'th')
(38, 'ao')
(39, 'sh')


In [71]:
one_hot_dict = {}
for idx, ph in enumerate(set(phoneme_array)):
  one_hot_dict[ph] = one_hot[idx,:]

In [72]:
one_hot_dict

{'aa': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]),
 'ae': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]),
 'ah': array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]),
 'ao': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0.]),
 'aw': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]),
 'ay': array([0., 0., 0., 0., 0., 0., 0.

In [107]:
final_audio = np.array(audio_array)
final_audio = final_audio.reshape(final_audio.shape[0], final_audio.shape[1])

In [ ]:
# fit model no training data
model = xgboost.XGBClassifier()
model.fit(final_audio, label_encoded_y)

In [92]:
phoneme_array[-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [98]:
np.array(phoneme_array).shape

(298634, 40)

In [105]:
audio_array[3].shape

(129, 1)

In [ ]:
max_len = 0

for i in audio_array:
  max_len = len(i)

In [109]:
final_audio.shape

(298634, 129)

In [110]:
len(phoneme_array)

298634

In [111]:
reverse_dict = {v:k for k,v in one_hot_dict.items()}

TypeError: ignored

In [117]:
int(np.where(phoneme_array[0]==1.)[0])

30

In [118]:
new_phoneme = []
for ph in phoneme_array:
  new_phoneme.append(int(np.where(ph==1.)[0]))

In [120]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(new_phoneme)

In [122]:
label_encoded_y = label_encoder.transform(new_phoneme)

In [124]:
label_encoded_y.shape

(298634,)